In [1]:
virus_fasta = "/home/laura/projects/virus-watch-data/virus_ref/uniques_noduplicates.fa"
virus_index = "/home/laura/projects/virus-watch-data/virus_ref/kallisto_aa/noduplicates_mouse.idx"
virus_t2g = "/home/laura/projects/virus-watch-data/virus_ref/uniques_noduplicates_t2g.txt"
host_fasta = "/home/laura/projects/virus-watch-data/mouse_ref/Mus_musculus.GRCm39.dna.primary_assembly.fa.gz"

bp = "PRJNA820450"
fastq_folder = f"/home/laura/projects/virus-watch-data/{bp}/raw"

out_folder = f"{fastq_folder}/../virus"

kallisto = "/home/laura/projects/kallisto/build/src/kallisto"
bustools = "/home/laura/projects/bustools/build/src/bustools"

Generate virus index d-listed against mouse:

In [5]:
# Generate virus reference index
!/usr/bin/time -v kallisto index \
    -t 60 \
    --aa \
    --d-list $host_fasta \
    -i $virus_index \
    $virus_fasta


[build] loading fasta file /home/laura/projects/virus-watch-data/virus_ref/uniques_noduplicates.fa
[build] k-mer length: 31
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Finished
CompactedDBG::build(): Estimated number of k-mers occurring at least once: 37641510
CompactedDBG::build(): Estimated number of minimizer occurring at least once: 7877811
CompactedDBG::filter(): Processed 87630084 k-mers in 296561 reads
CompactedDBG::filter(): Found 37508708 unique k-mers
CompactedDBG::filter(): Number of blocks in Bloom filter is 257317
CompactedDBG::construct(): Extract approximate unitigs (1/2)
CompactedDBG::construct(): Extract approximate unitigs (2/2)
CompactedDBG::construct(): Closed all input files

CompactedDBG::construct(): Splitting unitigs (1/2)

CompactedDBG::construct(): Splitting unitigs (2/2)
CompactedDBG::construct(): Before split: 2040591 uni

Get fastq files:

In [2]:
import os
import glob

In [3]:
fastqs = []
for filename in glob.glob(f"{fastq_folder}/*.fastq.gz"):
    fastqs.append(filename.split("/")[-1])
    
fastqs.sort()
fastqs

['SRR18496012_1.fastq.gz',
 'SRR18496012_2.fastq.gz',
 'SRR18496013_1.fastq.gz',
 'SRR18496013_2.fastq.gz',
 'SRR18496014_1.fastq.gz',
 'SRR18496014_2.fastq.gz',
 'SRR18496015_1.fastq.gz',
 'SRR18496015_2.fastq.gz',
 'SRR18496016_1.fastq.gz',
 'SRR18496016_2.fastq.gz',
 'SRR18496017_1.fastq.gz',
 'SRR18496017_2.fastq.gz',
 'SRR18496018_1.fastq.gz',
 'SRR18496018_2.fastq.gz',
 'SRR18496019_1.fastq.gz',
 'SRR18496019_2.fastq.gz']

In [4]:
len(fastqs)

16

In [5]:
samples = []
for fastq in fastqs:
    samples.append(fastq.split("_")[0])

In [6]:
samples = list(set(samples))
len(samples)

8

Align to PalmDB:  
Whitelist files (r1_RT_replace.txt and r1r2r3.txt) provided by Delaney for IGVF project (07/15/2023).

In [ ]:
%%time
for sample in samples:
    fastq1 = sample + "_1.fastq.gz"
    fastq2 = sample + "_2.fastq.gz"
    
    !mkdir -p $out_folder/$sample
    
    !/usr/bin/time -v kb count \
        --kallisto=$kallisto \
        --bustools=$bustools \
        --aa \
        --h5ad \
        -t 12 \
        -i $virus_index \
        -g $virus_t2g \
        -x SPLIT-Seq \
        -r $fastq_folder/whitelist/r1_RT_replace.txt \
        -w $fastq_folder/whitelist/r1r2r3.txt \
        -o $out_folder/$sample/ \
        $fastq_folder/$fastq1 $fastq_folder/$fastq2

In [23]:
## DEPRECATED
# %%time

# for sample in samples:
#     fastq1 = sample + "_1.fastq.gz"
#     fastq2 = sample + "_2.fastq.gz"
    
#     !mkdir -p $out_folder/$sample
    
#     !/usr/bin/time -v /home/laura/bin/kallisto bus \
#             -i $virus_index \
#             -o $out_folder/$sample/ \
#             --aa \
#             -t 60 \
#             -x split-seq \
#             $fastq_folder/$fastq1 $fastq_folder/$fastq2 \
#             &> $out_folder/$sample/kb_out.txt 
        
#     !$bustools sort -o $out_folder/$sample/output_sorted.bus $out_folder/$sample/output.bus

#     !$bustools count \
#         --genecounts \
#         --cm -m \
#         -o $out_folder/$sample/bustools_count/ \
#         -g $virus_t2g \
#         -e $out_folder/$sample/matrix.ec \
#         -t $out_folder/$sample/transcripts.txt \
#         $out_folder/$sample/output_sorted.bus